# Ejemplo Arbol de Decision

__Información__. Sea $X$ una variable aleatoria que obedece una distribución de probabilidad $p(X)$. Se define el contenido de información de un evento $x \in X$ como:\begin{equation} I(x)=-\log p(x)\end{equation}Notemos que eventos muy probables ($p(x)\approx 1$) tienen muy poco contenido de información, mientras que eventos muy poco probables ($p(x) \approx 0$) tienen mucho contenido de información.

__Entropía de Shannon__. Se define la entropía de Shannon de una variable aleatoria como la media (primer momento) de la información. Para el caso de una distribución discreta:\begin{equation}H=E_{x\sim p}[I(x)]=-\sum_{i} p(x)\log p(x),\end{equation}en donde $i$ corre sobre todos los elementos del espacio muestral.

In [1]:
from math import log
import operator 
def createDataSet():
#Esta funcion crea un dataset, header son los nombres de las columnas y la ultima columna es la etiqueta final
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    
    return dataSet, labels

In [2]:
def calcShannonEnt(dataSet):
    '''
    Calculo de la entropia de Shannon
       Input : 
           dataSet: el conjunto de datos
       Output: 
           shannonEnt: entropia de Shannon del dataset
    ''' 
    #las keys de labelCounts son las etiquetas
    numEntries = len(dataSet)
    labelCounts = {}
    
    #loop sobre las muestras
    for featVec in dataSet:
        #se obtiene la etiqueta de la muestra
        currentLabel = featVec[-1]
        
        #se crea la llave del diccionario en caso de que
        #la etiqueta no exista
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel] = 0
        
        #se suma al numero de elementos con esa etiqueta
        labelCounts[currentLabel] += 1
                
    shannonEnt = 0.0
    
    #p(x) viene dada por la frecuencia relativa de x
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
        
    return shannonEnt

In [3]:
# En esta celda obtenemos el dataset y  observamos la entropía de shannon
#PAra usos practicos, estamos suponiendo que la etiqueta final es pez/no pez
DataSet, labels = createDataSet()
H = calcShannonEnt(DataSet)
print(DataSet)
print(H)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0.970950594455


In [9]:
#Entre mas inhomogeneos los datos, mayor es la entropia

In [4]:
def splitDataSet(dataSet, axis, value):
    '''
    Hace la partición del dataset
       Inputs:
           dataSet: dataset
           axis   : atributo con el que haremos la particion
           value  : valor del atributo con el que haremos la particion
       Output:
           retDataSet: lista de muestras con el atributo 'axis'
                       y valor 'value' removidos (una vez que hemos 
                       partido sobre ese atributo, ya no lo necesitaremos)
    '''
    #al final, retDataSet es una lista de listas en donde en cada una de ellas esta 
    #excluido el atributo que seleccionamos para hacer la particion
    retDataSet = []
    
    #loop sobre las muestras    
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]         #corta la muestra antes del atributo
            reducedFeatVec.extend(featVec[axis+1:]) #pega lo que esta despues del atributo
            retDataSet.append(reducedFeatVec)       #añade el resultado a retDataSet
            
    return retDataSet

In [5]:
print('Dataset original')
print(DataSet,'\n')
print('muestras en donde el primer atributo vale 1:')
print(splitDataSet(DataSet,0,1),'\n')
print('muestras en donde el segundo atributo vale 0:')
print(splitDataSet(DataSet,1,0))

Dataset original
([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']], '\n')
muestras en donde el primer atributo vale 1:
([[1, 'yes'], [1, 'yes'], [0, 'no']], '\n')
muestras en donde el segundo atributo vale 0:
[[1, 'no']]


La idea es medir la entropía de Shannon antes de hacer la partición. Luego hacemos la partición y medimos la entropía final dada por la media de las entropías de cada uno de los subconjuntos. <em>La partición óptima será aquella en donde el cambio de entropía $|\Delta H|$ es mayor,</em> pues si después de partir el dataset tenemos datos más homogéneos, la entropía total se reduce al maximo.

In [6]:
def chooseBestFeatureToSplit(dataSet):
    '''
    Seleccion del mejor atributo para hacer la particion 
       Input:
           dataSet: dataset    
       Output:
           bestFeature: el atributo tal que al partir el dataset
                        se obtiene la mayor reduccion de la entropia
    '''
    
    numFeatures = len(dataSet[0]) - 1      
    
    #entropia inicial
    baseEntropy = calcShannonEnt(dataSet)
    print('entropia inicial: {:.4f}'.format(baseEntropy))
    
    #inicializacion de \Delta H
    bestInfoGain = 0.0
    bestFeature = -1
    
    #loop sobre los atributos
    for i in range(numFeatures):
        print('examinando atributo:',labels[i])
        #se obtiene una lista del valor del atributo i en cada una de las muestras
        featList = [example[i] for example in dataSet]   
        uniqueVals = set(featList)   #aqui se seleccionan los valores no repetidos
        newEntropy = 0.0
        
        #se hace la particion sobre el atributo i con cada uno de sus valores unicos
        for value in uniqueVals:
            print('\t estimando contribucion cuando el atributo vale {}'.format(value))
            subDataSet = splitDataSet(dataSet,i,value)
            #prob indica en cuantas muestras el atributo 'i' vale 'value'
            #a mayor cantidad de muestras, mayor sera el peso en \Delta H
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet) 
            
        print('\t media de la entropia de esta particion: {:.3f}'.format(newEntropy))
            
        #cambio de la entropia    
        infoGain = newEntropy-baseEntropy        
        print('\t Delta H: {:.3f}'.format(infoGain))
        
        #compara con la reduccion mas sustancial de Delta H
        if (infoGain < bestInfoGain):                    
            bestInfoGain = infoGain                      
            bestFeature = i
            
    return bestFeature #returns an integer

In [7]:
chooseBestFeatureToSplit(DataSet)

entropia inicial: 0.9710
('examinando atributo:', 'no surfacing')
	 estimando contribucion cuando el atributo vale 0
	 estimando contribucion cuando el atributo vale 1
	 media de la entropia de esta particion: 0.551
	 Delta H: -0.420
('examinando atributo:', 'flippers')
	 estimando contribucion cuando el atributo vale 0
	 estimando contribucion cuando el atributo vale 1
	 media de la entropia de esta particion: 0.800
	 Delta H: -0.171


0

In [8]:
def majorityCnt(classList):
    '''
    Decide la clase de los objetos del subdataset basado en la clase 
    con mayor frecuencia. Esta rutina se corre cuando al hacer la 
    particion nos hemos acabado todos los atributos
    Input:
        classList: lista de las labels de los objetos del subdataset
    Output:
        sortedClassCount: clase con mayor frecuencia en el subdataset
                          (esta sera la prediccion para ese subdataset)
    '''    
    #diccionario {class:numero_de_objetos}
    classCount={}
    
    #loop sobre los objetos del subdataset
    for vote in classList:
        #si la clase no esta, se introduce en el diccionario
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
            
    #aqui se determina la clase mayoritaria
    #python 2.7        
    #sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    #python 3+
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    
    return sortedClassCount[0][0]

In [9]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree 

In [10]:
myDat, labels = createDataSet()


In [11]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [12]:
print('labels antes de correr:',labels,'\n')

myTree = createTree(myDat,labels)

print('\n labels despues de correr:',labels,'\n')

print('myTree:\n',myTree)

('labels antes de correr:', ['no surfacing', 'flippers'], '\n')
entropia inicial: 0.9710
('examinando atributo:', 'no surfacing')
	 estimando contribucion cuando el atributo vale 0
	 estimando contribucion cuando el atributo vale 1
	 media de la entropia de esta particion: 0.551
	 Delta H: -0.420
('examinando atributo:', 'flippers')
	 estimando contribucion cuando el atributo vale 0
	 estimando contribucion cuando el atributo vale 1
	 media de la entropia de esta particion: 0.800
	 Delta H: -0.171
entropia inicial: 0.9183
('examinando atributo:', 'flippers')
	 estimando contribucion cuando el atributo vale 0
	 estimando contribucion cuando el atributo vale 1
	 media de la entropia de esta particion: 0.000
	 Delta H: -0.918
('\n labels despues de correr:', ['flippers'], '\n')
('myTree:\n', {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}})


# Para fines prácticos, usemos un dataset más complejo

## Titanic Dataset

In [13]:
#En esta celda leo el dataset de la pagina de stanford, la cual tiene labels "survive/not survive"

import pandas as pd

titanic = pd.read_csv('https://web.stanford.edu/class/archive' \
                      '/cs/cs109/cs109.1166/stuff/titanic.csv')
titanic.head(50)



,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


In [14]:
titanic.drop('Name', axis=1, inplace=True) # quita la columna names, no nos dice nada
titanic['Sex'] = pd.get_dummies(titanic['Sex']) #codificamos el genero de los pasajeros, 0 hombres/ 1 mujeres
data = titanic.values  # valores independientes del dataset
rows, columns = data.shape # columnas y filas

lowest_gini = ['Feature', 'Value', 0.50]  # Esta ocasion utilizamos el indice gini, otra metrica 
                                          # para calcular la homogeneidad de los datos
for column in range(1, columns):
    data_ = data[data[:, column].argsort()]
    
    for row in range(1, rows):
        L, R = data_[:row, 0], data_[row:, 0]

        gini_L = 1 - (sum(L==0)/len(L))**2 - (sum(L==1)/len(L))**2  #calcula la impureza de gini de la izquierda
        gini_R = 1 - (sum(R==0)/len(R))**2 - (sum(R==1)/len(R))**2  #calcula la impureza gini de la derecha

        gini = (len(L)*gini_L + len(R)*gini_R) / (len(L)+len(R))  #impureza de gini total
        
        if gini < lowest_gini[2]: 
            lowest_gini = [titanic.columns[column], data_[row, column], gini]  #nos dice si hacer un split dependiendo
                                                                                #de la impureza de gini
print('Best split-feature     :', {lowest_gini[0]})
print('Value to split on      :', {lowest_gini[1]})
print('Weighted gini impurity :', {round(lowest_gini[2], 5)})

('Best split-feature     :', set(['Pclass']))
('Value to split on      :', set([1.0]))
('Weighted gini impurity :', set([0.0]))


# Veamos que dice ScikitLearn

In [15]:
from sklearn.tree import DecisionTreeClassifier

X = titanic.drop('Survived', axis=1).values
y = titanic['Survived'].values

model = DecisionTreeClassifier(criterion = 'gini', max_depth=2)
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### veamos como se ve
Es posible que tengan que instalar graphviz, se puede hacer con el comando de la siguiente celda

In [66]:
#import sys
#! {sys.executable} - m pip install graphviz

In [16]:
from sklearn import tree
from graphviz import Source

X = titanic.drop('Survived', axis=1).values
y = titanic['Survived'].values

model = tree.DecisionTreeClassifier(criterion= 'gini', max_depth=2)
model.fit(X, y)

graph = Source(tree.export_graphviz(model, out_file=None,
                                    feature_names=titanic.columns[1:],
                                    class_names=['Not survived', 'Survived'],
                                    filled=True, rounded=True))
graph

ImportError: No module named graphviz

Era más probable que no sobreviviera al hundimiento del Titanic. Como podemos ver en la primera figura, el árbol de decisión comienza a dividirse sobre el sexo, 0 = masculino, 1 = femenino, luego pregunta si la persona es un niño (Edad <= 13) en caso de que el modelo fuera para el etiqueta verdadera (masculino), por lo que si seguimos esta rama, comienza preguntando "¿Es la persona masculina?" Si es así, pregunta "¿Es la persona un niño (menor de 13 años)? Y luego llega a un nodo de la hoja, donde si la persona es un hombre y un niño, sobrevivió, si no es un niño, entonces no sobrevivió.

De manera similar, si optamos por la rama falsa, que nos dice que la persona es una mujer, entonces pregunta cuál era la clase que tenía la mujer (1ra, 2da o 3ra clase), si la persona era de primera clase, entonces ella sobrevive, en el otro caso no sobrevivió.

De la información de este gráfico podemos decir lo siguiente

La clase survived está compuesta por niños y mujeres de 1ra clase, cada uno con 41 y 170 muestras respectivamente, en el otro caso, la clase de not survived nos dice que casi todos los hombres no sobrevivieron (532) y que las mujeres de las clases segunda y tercera tampoco sobrevivieron. Tiempos muy tristes aquí.

